In [25]:
from qiskit_aer import AerSimulator
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import MCMT
import time
import hashlib

def classical_pow(target_zeros, max_nonce):
    steps = 0
    for nonce in range(max_nonce):
        steps += 1
        hash_result = hashlib.sha256(str(nonce).encode()).hexdigest()
        if hash_result.startswith('0' * target_zeros):
            return nonce, hash_result, steps
    return None, None, steps

def grovers_circuit(num_qubits, target_state):
    qc = QuantumCircuit(num_qubits, num_qubits)  # Add a classical register for measurements

    # Initialize in the uniform superposition
    qc.h(range(num_qubits))

    # Apply a single iteration of Grover's algorithm
    # Oracle: flips the sign of the target state
    qc.h(target_state)
    qc.z(target_state)
    qc.h(target_state)

    # Diffuser
    qc.h(range(num_qubits))
    qc.x(range(num_qubits))
    qc.h(num_qubits - 1)
    qc.append(MCMT('x', num_ctrl_qubits=num_qubits - 1, num_target_qubits=1), range(num_qubits))
    qc.h(num_qubits - 1)
    qc.x(range(num_qubits))
    qc.h(range(num_qubits))

    # Measurement
    qc.measure(range(num_qubits), range(num_qubits))

    return qc

def quantum_pow(target_zeros, max_nonce, num_qubits):
    # Construct a quantum circuit for Grover's algorithm
    # Ensure target_state is within the range representable by num_qubits
    target_state = min(int('1' * target_zeros, 2), 2**num_qubits - 1)
    qc = grovers_circuit(num_qubits, target_state)
    
    # Simulate the quantum circuit
    simulator = AerSimulator()
    transpiled_circuit = transpile(qc, simulator)
    result = simulator.run(transpiled_circuit, shots=1).result()
    
    # Extract the result and convert to a nonce
    nonce = int(result.get_counts().most_frequent(), 2)
    return nonce, hashlib.sha256(str(nonce).encode()).hexdigest(), 1  # 1 Grover iteration

# Parameters
target_zeros = 3
max_nonce = 2**10
num_qubits = 10

# Classical PoW
nonce, hash_result, classical_steps = classical_pow(target_zeros, max_nonce)
print(f"Classical PoW: Nonce = {nonce}, Hash = {hash_result}, Steps = {classical_steps}")

# Quantum PoW
nonce, hash_result, quantum_steps = quantum_pow(target_zeros, max_nonce, num_qubits)
print(f"Quantum PoW: Nonce = {nonce}, Hash = {hash_result}, Steps = {quantum_steps}")

# Compare steps
speedup = classical_steps / quantum_steps
print(f"Speedup (in terms of steps): {speedup:.2f}x")


Classical PoW: Nonce = 886, Hash = 000f21ac06aceb9cdd0575e82d0d85fc39bed0a7a1d71970ba1641666a44f530, Steps = 887
Quantum PoW: Nonce = 907, Hash = c8c9cad7b920b50f713830b8dc55f59fffbbad98335d9f30e0bca8fab5dfeedd, Steps = 1
Speedup (in terms of steps): 887.00x
